# Popular 10000 movies listed on TMDb

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from jupyterthemes import jtplot
jtplot.style(theme='onedork', figsize=(15, 9))

In [15]:
cast_df = pd.read_csv('../data/popular_10000_movies/cast_dataset.csv', index_col='id')
cast_df

,cast
id,
536437,"[{'cast_id': 880, 'name': 'Ben Affleck', 'char..."
1074034,"[{'cast_id': 1244, 'name': 'Jonathan Rhys Meye..."
672,"[{'cast_id': 10980, 'name': 'Daniel Radcliffe'..."
870518,"[{'cast_id': 1392137, 'name': 'Margaret Qualle..."
663712,"[{'cast_id': 1882502, 'name': 'Lauren LaVera',..."
...,...
26141,"[{'cast_id': 23346, 'name': 'Armand Assante', ..."
494031,"[{'cast_id': 1482727, 'name': 'Diana Angelson'..."
483104,"[{'cast_id': 53485, 'name': 'Rose McIver', 'ch..."


In [16]:
crew_df = pd.read_csv('../data/popular_10000_movies/crew_dataset.csv', index_col='id')
crew_df

,crew
id,
536437,"[{'crew_id': 2294, 'name': 'Robert Rodriguez',..."
1074034,"[{'crew_id': 58267, 'name': 'Matthew Cassel', ..."
672,"[{'crew_id': 491, 'name': 'John Williams', 'de..."
870518,"[{'crew_id': 46805, 'name': 'Johnathan Dorfman..."
663712,"[{'crew_id': 83331, 'name': 'Steven Barton', '..."
...,...
26141,"[{'crew_id': 639, 'name': 'Pieter Jan Brugge',..."
494031,"[{'crew_id': 1278976, 'name': 'Daniel Anghelce..."
483104,"[{'crew_id': 20905, 'name': 'Melissa DeLizia',..."


In [28]:
keywords_df = pd.read_csv('../data/popular_10000_movies/keywords_dataframe.csv', index_col='id')
keywords_df.drop(['Unnamed: 0'], axis=1, inplace=True)
keywords_df

,keywords
id,
640146,"[{'id': 1701, 'name': 'hero'}, {'id': 3126, 'n..."
934433,"[{'id': 242, 'name': 'new york city'}, {'id': ..."
879444,"[{'id': 779, 'name': 'martial arts'}, {'id': 9..."
1036561,"[{'id': 779, 'name': 'martial arts'}, {'id': 9..."
677179,"[{'id': 276, 'name': 'philadelphia, pennsylvan..."
...,...
987337,[]
10081,"[{'id': 836, 'name': 'resistance'}, {'id': 214..."
9771,"[{'id': 1480, 'name': 'baseball'}, {'id': 1482..."


In [18]:
mov_df = pd.read_csv('../data/popular_10000_movies/movies_dataset.csv', index_col='id')
mov_df

,genre_ids,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres
id,,,,,,,,,
385687,"[28, 80, 53]",en,Over many missions and against impossible odds...,4654.279,2023-05-17,Fast X,7.3,2093,"Action, Crime, Thriller"
697843,"[28, 53]",en,Tasked with extracting a family who is at the ...,2494.010,2023-06-09,Extraction 2,7.7,910,"Action, Thriller"
603692,"[28, 53, 80]",en,"With the price on his head ever increasing, Jo...",1920.127,2023-03-22,John Wick: Chapter 4,7.9,3344,"Action, Thriller, Crime"
569094,"[28, 12, 16, 878]",en,"After reuniting with Gwen Stacy, Brooklyn’s fu...",2013.795,2023-05-31,Spider-Man: Across the Spider-Verse,8.6,1796,"Action, Adventure, Animation, Science Fiction"
502356,"[16, 10751, 12, 14, 35]",en,"While working underground to fix a water main,...",1539.037,2023-04-05,The Super Mario Bros. Movie,7.8,5165,"Animation, Family, Adventure, Fantasy, Comedy"
...,...,...,...,...,...,...,...,...,...
15017,"[35, 10749]",en,A young professional woman (Simpson) unwitting...,11.077,2007-12-21,Blonde Ambition,4.6,97,"Comedy, Romance"
79509,"[28, 53, 35]",en,Josh meets a young woman who shortly afterward...,11.076,1990-03-22,The Ambulance,5.9,85,"Action, Thriller, Comedy"
13370,"[35, 10402]",en,Director Christopher Guest reunites the team f...,11.075,2003-04-16,A Mighty Wind,6.8,206,"Comedy, Music"


In [29]:
tmdb_df = mov_df.join(cast_df, on='id')
tmdb_df = tmdb_df.join(crew_df, on='id')
tmdb_df = tmdb_df.join(keywords_df, on='id')
tmdb_df

,genre_ids,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords
id,,,,,,,,,,,,
385687,"[28, 80, 53]",en,Over many missions and against impossible odds...,4654.279,2023-05-17,Fast X,7.3,2093,"Action, Crime, Thriller","[{'cast_id': 12835, 'name': 'Vin Diesel', 'cha...","[{'crew_id': 1302, 'name': 'Susie Figgis', 'de...","[{'id': 9663, 'name': 'sequel'}, {'id': 9748, ..."
697843,"[28, 53]",en,Tasked with extracting a family who is at the ...,2494.010,2023-06-09,Extraction 2,7.7,910,"Action, Thriller","[{'cast_id': 74568, 'name': 'Chris Hemsworth',...","[{'crew_id': 950, 'name': 'Pietro Scalia', 'de...","[{'id': 3070, 'name': 'mercenary'}, {'id': 966..."
603692,"[28, 53, 80]",en,"With the price on his head ever increasing, Jo...",1920.127,2023-03-22,John Wick: Chapter 4,7.9,3344,"Action, Thriller, Crime","[{'cast_id': 6384, 'name': 'Keanu Reeves', 'ch...","[{'crew_id': 3615, 'name': 'Manfred Banach', '...","[{'id': 242, 'name': 'new york city'}, {'id': ..."
569094,"[28, 12, 16, 878]",en,"After reuniting with Gwen Stacy, Brooklyn’s fu...",2013.795,2023-05-31,Spider-Man: Across the Spider-Verse,8.6,1796,"Action, Adventure, Animation, Science Fiction","[{'cast_id': 587506, 'name': 'Shameik Moore', ...","[{'crew_id': 7624, 'name': 'Stan Lee', 'depart...","[{'id': 2858, 'name': 'sacrifice'}, {'id': 328..."
502356,"[16, 10751, 12, 14, 35]",en,"While working underground to fix a water main,...",1539.037,2023-04-05,The Super Mario Bros. Movie,7.8,5165,"Animation, Family, Adventure, Fantasy, Comedy","[{'cast_id': 73457, 'name': 'Chris Pratt', 'ch...","[{'crew_id': 70851, 'name': 'Jack Black', 'dep...","[{'id': 282, 'name': 'video game'}, {'id': 690..."
...,...,...,...,...,...,...,...,...,...,...,...,...
15017,"[35, 10749]",en,A young professional woman (Simpson) unwitting...,11.077,2007-12-21,Blonde Ambition,4.6,97,"Comedy, Romance","[{'cast_id': 52052, 'name': 'Jessica Simpson',...","[{'crew_id': 7413, 'name': 'Mark Irwin', 'depa...","[{'id': 6593, 'name': 'stripper'}, {'id': 9713..."
79509,"[28, 53, 35]",en,Josh meets a young woman who shortly afterward...,11.076,1990-03-22,The Ambulance,5.9,85,"Action, Thriller, Comedy","[{'cast_id': 21315, 'name': 'Eric Roberts', 'c...","[{'crew_id': 13670, 'name': 'Jacques Haitkin',...","[{'id': 242, 'name': 'new york city'}, {'id': ..."
13370,"[35, 10402]",en,Director Christopher Guest reunites the team f...,11.075,2003-04-16,A Mighty Wind,6.8,206,"Comedy, Music","[{'cast_id': 12438, 'name': 'Bob Balaban', 'ch...","[{'crew_id': 3032, 'name': 'Robert Leighton', ...","[{'id': 11800, 'name': 'mockumentary'}, {'id':..."
